# 전통적 시계열 모델링

   ## 유통매장 수요량 예측

* 비즈니스 현황
    * 고객사는 A 유통회사의 a 매장 입니다.
    * a 매장에서 주력상품인 a01에 대한 재고 최적화를 위해 수요량을 예측하고자 합니다.
    * 최근 경쟁사의 매장이 가까운 거리에 오픈하였고, 유사한 상품에 대한 공격적인 마케팅을 펼치고 있습니다. 
* 발주 최적화를 위한 수요량 예측
    * 일마감 이후, 발주량을 결정할 때, 예측된 수요량이 필요합니다.
    * 발주후 입고까지는 2일의 기간이 걸립니다. 
    * 예를 들면 
        * 2019년 6월 1일 저녁 10시 일마감 직후, 6월 3일의 수요량을 예측해야 합니다.


![](https://www.artefact.com//wp-content/uploads/2021/08/GettyImages-1295864156-scaled.jpg)

# 1.환경 준비

## (1) Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as spst
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

## (2) Data Loading

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/retail_demand2.csv'
data = pd.read_csv(path)
data = data.loc[(data['date']>= '2013-06-01') & (data['date']< '2015-07-01')].reset_index(drop = True)
data.head(10)

* 변수 설명

    * date : 날짜
    * item : 상품코드 (여기서는 한가지 상품만 있음)
    * sales : A유통회사 a 매장 판매량 ==> target
    * tot_sales : A유통회사 전체 판매량
    * comp_sales : 인근에 위치한 B유통회사 b 매장 판매량

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(data['sales'])
plt.grid()
plt.show()

## (3) 함수 생성 

* 잔차분석

In [ ]:
def residual_diag(residuals, lags = 20) :

    print('* 정규성 검정(> 0.05) : ', round(spst.shapiro(residuals)[1],5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1],5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

# 2.데이터 준비 

## (1) 시간정보 인덱스 만들기
* 날짜 타입으로 변환
* 날짜를 인덱스로
* freq 지정하기

### 1) 날짜 타입으로 변경하기

### 2) 날짜를 인덱스로 변환하기

### 3) 날짜단위 지정하기 : freq

* **분석 단위**를 어떻게 가져갈 것인가와 관련이 있습니다.
* 시계열 데이터를 **일정한 시간 간격**으로 만들어 줍니다.
* 인덱스 조회시, 마지막에 있는 **freq** 옵션

## (2) y 만들기
2일후의 판매량을 예측해야 합니다.

## (3) NaN 조치 
* y 생성으로 인해 NaN이 발생되었습니다. 어떻게 조치해야 할까요?

## (4) 가변수화

## (5) 데이터 분할

* Cross Validation : 데이터의 마지막 6개월을 6등분해서 교차 검증해 봅시다.
    * fold : 6
    * validation size : 1개월(30일)

### 1) x, y 나누기

### 2) 시계열 데이터 분할

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# validation set size
val_size = 
nfold = 

tscv = TimeSeriesSplit( )

# 4.모델링 : Baseline Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import * 

## (1) Cross Validation

* for loop를 이용하여 Cross Validation을 수행하시오.
* 알고리즘은 LinearRegression을 이용하여 default로 사용

In [ ]:
model = 

In [ ]:
rmse, mae, mape = [],[],[]
residuals = []
pred = []

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_val, y_val = x.iloc[val_index], y.iloc[val_index]

    # 학습
    model.fit(x_train, y_train)

    # 예측
    pr = model.predict(x_val)
    pred += list(pr)

    # 평가
    rmse.append(mean_squared_error(y_val, pr, squared = False))
    mae.append(mean_absolute_error(y_val, pr))
    mape.append(mean_absolute_percentage_error(y_val, pr))

    # 잔차 : 각 fold의 결과를 리스트로 변환하여 추가
    residuals += list(y_val - pr)

## (2) 예측 결과 평가

### 1) RMSE, MAE, MAPE

In [ ]:
print('RMSE : ',round(np.mean(rmse),4))
print('MAE  : ',round(np.mean(mae),4))
print('MAPE : ',round(np.mean(mape),4))

### 2) 그래프 비교

In [ ]:
n = val_size * nfold
pred = pd.Series(pred, index = y[-n:].index)

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y[:-n], label = 'train')
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

# 5.평가 : 잔차분석

## (1) 시각화

* 잔차에 대해 라인차트, 히스토그램 등을 그려봅시다.

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(residuals)
plt.axhline(0, color = 'r', ls = '--')
plt.axhline(np.mean(residuals), color = 'g', ls = '--')
plt.show()

## (2) ACF, PACF

* acf, pacf 그림을 그려 자기 상관성 여부를 판단해 봅시다.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
lags = 20

fig,ax = plt.subplots(1,2, figsize = (15,5))
plot_acf(residuals, lags = lags, ax = ax[0])
plot_pacf(residuals, lags = lags, ax = ax[1])
plt.show()

## (3) 검정

* 정규성 검정, 정상성 검정을 수행하고 판정해 봅시다.

In [ ]:
from scipy import stats
import statsmodels.api as sm

* 정규성 검정 : Shapiro-Wilk 검정

In [ ]:
stats.shapiro(residuals)[1]

* 정상성 검정 : ADF 검정

In [ ]:
sm.tsa.stattools.adfuller(residuals)[1]

# 6.모델링 : ARIMA

In [ ]:
from sklearn.metrics import *

## (1) y 값 살펴보기

In [ ]:
residual_diag(y, lags = 20)

## (2) 모델링 : 초기모델

* p, d, q 값을 어떻게 정해야 할까요?
* AR의 p 차수와 MA q 차수 모두 값이 필요해 보입니다. 일단 1, 1을 지정합시다.

In [ ]:
train = y[:-30]
val = y[-30:]

### 1) 학습

* sm.tsa.SARIMAX(train, order=(p,d,q)).fit()
    * 모델 선언시 train이 포함
    * .fit()으로 학습.

In [ ]:
model1 = 

### 2) 평가

#### ① 잔차진단

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

#### ③ Validation

시계열 데이터로 실제값과 예측값에 대해 비교하여 그래프를 그려봅시다.

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(train[-120:], label='train')
plt.plot(pred, label = 'forecast')
plt.plot(val, label = 'val')
plt.legend()
plt.show()

## (3) 하이퍼파라미터 튜닝

### 1) 학습

In [ ]:
from itertools import product

* 값의 범위 지정

In [ ]:
# product 함수를 이용하여 값의 조합을 구성
p = [1,2,3,4]
q = [1,2,3,4]
d = [1,2]
iter = list(product(p,d,q))
iter

In [ ]:
# 
mae, aic = [],[]
for i in iter :
    model_fit = 
    pred = 
    mae.append( mean_absolute_error(val, pred))
    aic.append(model_fit.aic)
    print(i)

In [ ]:
result = pd.DataFrame({'params(p,d,q)' : iter, 'mae' : mae, 'aic':aic})

display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

In [ ]:
model2_1 = 
model2_2 = 

### 2) 평가

#### ① 잔차진단

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

#### ③ Validation(그래프 비교)


## (4) Cross Validation

### 1) 학습

In [ ]:
rmse, mae, mape, aic = [],[],[],[]
residuals = []
preds = []
p,d,q = 2,2,3

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    train = y[train_index]
    val = y[val_index]

    # 학습
    model = sm.tsa.SARIMAX(train, order=(p,d,q)).fit()

    # 예측
    pred = model.forecast(val_size)
    preds += list(pred)

    # 잔차 저장
    residuals += list(model.resid)

    # 평가
    rmse.append(mean_squared_error(val, pred, squared = False))
    mae.append(mean_absolute_error(val, pred))
    mape.append(mean_absolute_percentage_error(val, pred))
    aic.append(model.aic)

In [ ]:
print('RMSE : ',round(np.mean(rmse),4))
print('MAE  : ',round(np.mean(mae),4))
print('MAPE : ',round(np.mean(mape),4))

### 2) 평가

#### ① 잔차진단

* residual_diag

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

#### ③ Validation

In [ ]:
n = nfold * val_size

In [ ]:
preds = pd.Series(preds, index = y[-n:].index)
print('MAE :', mean_absolute_error(y[-n:], preds))

plt.figure(figsize = (20,8))
plt.plot(y[-200:], label = 'train')
plt.plot(y[-n:], label = 'val')
plt.plot(preds, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

# 7.모델링 : SARIMA

## (1) 모델링 : 초기모델

In [ ]:
train = y[:-30]
val = y[-30:]

### 1) 학습

In [ ]:
model1 = sm.tsa.SARIMAX(train, order=(2,2,3), seasonal_order = (1,1,1,7)).fit()

### 2) 평가

#### ① 잔차진단

#### ② AIC

#### ③ Validation(그래프 비교)


## (2) 하이퍼파라미터 튜닝

### 1) 학습

In [ ]:
from itertools import product

### 2) 평가

#### ① 잔차진단

#### ② AIC

#### ③ Validation(그래프 비교)


## (3) Cross Validation

### 1) 학습

### 2) 평가1

#### ① 잔차진단

#### ② AIC

#### ③ Validation(그래프 비교)


# 8.모델링 : SARIMAX

## (1) 모델링

### 1) 학습

In [ ]:
val_size = 30
x_train, y_train = x[:-val_size], y[:-val_size]
x_val, y_val = x[-val_size:], y[-val_size:]

In [ ]:
x_train.shape, y_train.shape

### 2) 평가

#### ① 잔차진단

* residual_diag

#### ② AIC

#### ③ Validation (그래프비교)
SARIMAX 모델을 생성하고, 예측할 때는 exog=x_val 옵션이 들어가야 함.

In [ ]:
pred = model3_1.forecast(30,  exog=x_val)
mean_absolute_error(val, pred)

## (2) Cross Validation

### 1) 학습

### 2) 평가

#### ① 잔차진단

#### ② AIC

#### ③ Validation(그래프 비교)


-------